In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from multirtd.dubins_model import dubins_traj, dubins_traj_new, linearize_dynamics

%load_ext autoreload
%autoreload 2

### Monte carlo

In [ ]:
# Initial state (no uncertainty)
x0 = np.array([0, 0, 0])  # [x, y, theta]

# Monte carlo over 1000 samples
dt = 0.1
N_MC = 1000
K = 100  # Number of timesteps
sigma = 0.03  # rotational velocity sigma

u = np.zeros((K, 2))
u[:, 0] = 1  # constant velocity

x_samples = []
for i in range(N_MC):
    u[:, 1] = np.random.normal(0, sigma, K)  # random rotational velocity
    x = dubins_traj(x0, u, dt)
    x_samples.append(x)

# Plot the trajectory
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
for x in x_samples:
    ax1.plot(x[:,0], x[:,1], 'b', alpha=0.1)
    ax2.scatter(x[-1,0], x[-1,1])
plt.axis('equal')
plt.show()

In [ ]:
dt = 1.0
K = 30
sigma = 0.3

from multirtd.dubins_model import linearize_dynamics
# # Linearized dynamics
# def linearize_dynamics(x, u, dt):
#     A = np.eye(3)
#     A[0, 2] = -u[0] * dt * np.sin(x[2])
#     A[1, 2] = u[0] * dt * np.cos(x[2])
#     B = np.zeros((3, 2))
#     B[0, 0] = dt * np.cos(x[2])
#     B[1, 0] = dt * np.sin(x[2])
#     B[2, 1] = dt
#     return A, B

u = np.zeros((K, 2))
u[:, 0] = 1  # constant velocity
#u[:, 1] = np.random.normal(0, sigma, K)  # random rotational velocity
u[:, 1] = 0.1
x = dubins_traj_new(x0, u, dt)

# Compare linearized dynamics with actual dynamics
x_lin = np.zeros((K, 3))
x_lin[0] = x0
for i in range(K-1):
    A, B = linearize_dynamics(x[i], u[i], dt)
    x_lin[i+1] = A @ x_lin[i] + B @ u[i]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.axis('equal')
ax1.plot(x[:,0], x[:,1], 'b', label='Actual')
ax1.plot(x_lin[:,0], x_lin[:,1], 'r', label='Linearized')

In [ ]:
dt = 2.0
K = 30
u = np.zeros((K, 2))
u[:, 0] = 1 
u[:, 1] = 0.01

# Compare dubins_traj and dubins_traj_new
x = dubins_traj(x0, u, dt)
x_new = dubins_traj_new(x0, u, dt)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.plot(x[:,0], x[:,1], 'b')
ax1.plot(x_new[:,0], x_new[:,1], 'r')

### Akshay's reachability

### RRBT Propagation

Relies on nominal trajectory - linearizes dubin's dynamics to get $(A_t, B_t)$ around nominal trajectory

Uses ranging beacon measurement setup, linearized measurement model around nominal trajectory to get $(C_t)$

Uses LQR to get $(K_t)$

In [ ]:
from multirtd.dubins_reachability import generate_robot_matrices

In [ ]:
def system_matrices(x, u, Q_lqr, R_lqr, dt):
    A, B = linearize_dynamics(x, u, dt)
    Q = Q_lqr
    R = R_lqr
    return A, B, Q, R

In [ ]:
dt = 0.1
N = 30
x0 = np.array([0, 0, 0])
u_nom = np.zeros((N, 2))
u_nom[:, 0] = 1
u_nom[:, 1] = 0.1
x_nom = dubins_traj(x0, u_nom, dt)



### Robotics toolbox

In [ ]:
from roboticstoolbox.mobile import Bicycle, RandomPath

V = np.diag([0.2, np.radians(0.5)])
robot = Bicycle(covar=V, dt=0.1)
robot.control = RandomPath(workspace=10)

In [ ]:
N_MC = 100
x_samples = []
fig, ax = plt.subplots(figsize=(5, 5))
for i in range(N_MC):
    x = robot.run(10)
    x_samples.append(x)
    ax.plot(x[:, 0], x[:, 1])
ax.axis('equal')

### Closed loop simulation

In [ ]:
def diff_drive(vl, vr, sigma, N, d, dt):
    """Diff drive model with uncertainty in the wheel velocities.

    d : distance between wheels

    """
    x = np.zeros((N,3))
    for i in range(N-1):
        noise = np.random.normal(0, sigma, 2)
        dx = np.array([0.5 * (vl + vr + noise[0] + noise[1]) * np.cos(x[i,2]),
                       0.5 * (vl + vr + noise[0] + noise[1]) * np.sin(x[i,2]),
                       (vr - vl + noise[0] - noise[1]) / d])
        x[i+1] = x[i] + dx * dt
    return x

In [ ]:
N_MC = 100
N = 100
d = 0.1
dt = 0.1
sigma = 0.01
vl = 0.15
vr = 0.2

x_MC = np.zeros((N_MC, N, 3))
for i in range(N_MC):
    x = diff_drive(vl, vr, sigma, N, d, dt)
    x_MC[i] = x
    plt.plot(x[:,0], x[:,1])
plt.axis('equal')
plt.show()

In [ ]:
for i in range(N_MC):
    plt.plot(x_MC[i,0:N:10,0], x_MC[i,0:N:10,1], 'b.', markersize=1)
    #plt.plot(x_MC[i,-1,0], x_MC[i,-1,1], 'b.', markersize=1)
plt.axis('equal')
plt.show()

In [ ]:
v = (vl + vr) / 2
w = (vr - vl) / d
u_nom = np.array([[v, w]] * N)
x_nom = dubins_traj(x0, u_nom, dt)

plt.plot(x_nom[:, 0], x_nom[:, 1])
plt.axis('equal')
plt.show()

In [ ]:
# Error state linearization

x_err = np.zeros((N, 3))

for i in range(N):
    A, B = linearize_dynamics(x_nom[i], u_nom[i], dt)

In [ ]:
from matplotlib.patches import Ellipse

def plot_ellipse(ax, c, Sigma):
    """Plot 0.95 confidence ellipse from center and covariance matrix
    
    """
    vals, vecs = np.linalg.eig(Sigma)
    order = vals.argsort()[::-1]
    vals = vals[order]
    vecs = vecs[:, order]
    theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))
    width, height = 2 * np.sqrt(5.991) * np.sqrt(vals)
    ellip = Ellipse(xy=c, width=width, height=height, angle=theta, alpha=0.5)
    ax.add_artist(ellip)


In [ ]:
# RRBT

Sigma = np.zeros((3, 3))
Q = sigma**2 * np.eye(2)

fig, ax = plt.subplots(figsize=(5, 5))
for i in range(1,N):
    A, B = linearize_dynamics(x_nom[i], u_nom[i], dt)
    C = np.array([[0.5*dt*np.cos(x_nom[i,2]), 0.5*dt*np.cos(x_nom[i,2])], 
                  [0.5*dt*np.sin(x_nom[i,2]), 0.5*dt*np.sin(x_nom[i,2])], 
                  [-dt/d, dt/d]])
    Sigma = A @ Sigma @ A.T + C @ Q @ C.T
    if i % 10 == 0:
        plot_ellipse(ax, x_nom[i], Sigma[:2,:2])

for i in range(N_MC):
    ax.plot(x_MC[i,0:N:10,0], x_MC[i,0:N:10,1], 'b.', markersize=1)
ax.axis('equal')

ax.set_xlim(-0.5, 0.5)
ax.set_ylim(0, 0.8)

In [ ]:
Sigma

In [ ]:
vals, vecs = np.linalg.eig(Sigma)
order = vals.argsort()[::-1]
vals = vals[order]
vecs = vecs[:, order]
vecs[:, 0][::-1]

In [ ]:
theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))

In [ ]:
DT = 0.1  # Time step
N = 15  # Number of timesteps

# Nominal trajectory
x0 = np.array([0, 0, 0])
u_nom = np.array([[1, 0.1]] * N)
x_nom = dubins_traj(x0, u_nom, DT)

# Actual trajectory
sigma = np.diag([0.001, 0.001, 0.01])
x = dubins_traj(x0, u_nom, DT, sigma)


# Plot trajectory
plt.plot(x_nom[:, 0], x_nom[:, 1])
plt.plot(x[:, 0], x[:, 1])
plt.axis('equal')
plt.show()

In [ ]:
def measurement(x, Sigma):
    """Measurement function
    
    (x,y,theta) -> (x,y) + noise
    
    """
    return x[:2] + np.random.multivariate_normal(np.zeros(2), Sigma)

In [ ]:
# Kalman filter
C = np.array([[1, 0, 0], [0, 1, 0]])  # Measurement matrix